In [1]:
import pandas as pd
import jieba
import re
from gensim.models import word2vec 

In [2]:
# 分词
def tokenizer(text): 
    return [word for word in jieba.lcut(text) if word not in stop_words]

In [3]:
# 去停用词
def get_stop_words():
    file_object = open('data/stopwords.txt',encoding='utf-8')
    stop_words = []
    for line in file_object.readlines():
        line = line[:-1]
        line = line.strip()
        stop_words.append(line)
    return stop_words

In [4]:
traindata = pd.read_csv('data/train.tsv', sep='\t')
validata = pd.read_csv('data/validation.tsv', sep='\t')
traindata.head()

,index,label,text
0,0,0,备胎是硬伤！
1,1,0,要说不满意的话，那就是动力了，1.5自然吸气发动机对这款车有种小马拉大车的感觉。如今天气这么...
2,2,0,油耗显示13升还多一点，希望慢慢下降。没有倒车雷达真可恨
3,3,0,空调不太凉，应该是小问题。
4,4,0,1、后排座椅不能平放；2、科技感不强，还不如百万帝豪，最希望增加车联网的车机。像你好博越一样...


In [5]:
totaldata = pd.concat([traindata, validata])

In [6]:
totaldata.to_csv('data/text.tsv', columns=['text'], index=0)
text = pd.read_csv('data/text.tsv')

In [7]:
stop_words = get_stop_words()
stop_words[0:5]

['!', '"', '#', '$', '%']

In [8]:
text_cut = []
for row in text.itertuples():
    seg = tokenizer(row[1])
    text_cut.append(seg)
print(text_cut[0:5])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.259 seconds.
Prefix dict has been built successfully.


[['备胎', '硬伤'], ['要说', '满意', '动力', '1.5', '自然', '吸气', '发动机', '这款', '车', '有种', '小马拉', '大车', '感觉', '天气', '热', '上路', '肯定', '得开', '空调', '开', '动力', '感觉', '不给力', '空调', '制冷', '效果', '不错'], ['油耗', '显示', '13', '升', '多一点', '希望', '慢慢', '下降', '倒车', '雷达', '真', '可恨'], ['空调', '不太凉'], ['1', '后排', '座椅', '平放', '2', '科技', '感不强', '百万', '帝豪', '希望', '增加', '车', '联网', '车机', '你好', '博越', '3', '全景', '摄像头', '晚上', '用处']]


In [9]:
text_concat=[]
for seg in text_cut:
    seg_concat=[" ".join(word for word in seg)]
    text_concat.append(seg_concat)
print(text_concat[0:5])

[['备胎 硬伤'], ['要说 满意 动力 1.5 自然 吸气 发动机 这款 车 有种 小马拉 大车 感觉 天气 热 上路 肯定 得开 空调 开 动力 感觉 不给力 空调 制冷 效果 不错'], ['油耗 显示 13 升 多一点 希望 慢慢 下降 倒车 雷达 真 可恨'], ['空调 不太凉'], ['1 后排 座椅 平放 2 科技 感不强 百万 帝豪 希望 增加 车 联网 车机 你好 博越 3 全景 摄像头 晚上 用处']]


In [10]:
corpus = pd.DataFrame(data=text_concat)

In [11]:
corpus.to_csv('data/corpus.tsv', header=0, index=0)

In [12]:
sentences = word2vec.LineSentence('data/corpus.tsv')
model = word2vec.Word2Vec(sentences, min_count=5)# 待考虑

In [13]:
pairs = [
    ('备胎', '硬伤'),  
    ('车', '用处'),   
    ('百万', '空调'), 
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2,model.wv.similarity(w1, w2)))

'备胎'	'硬伤'	0.52
'车'	'用处'	0.27
'百万'	'空调'	0.11


In [14]:
vector = model.wv['车']

In [15]:
word_vectors = model.wv

In [16]:
model.save("comment.model")

In [17]:
new_model = word2vec.Word2Vec.load('comment.model')
new_model.wv.save_word2vec_format('data/myvector.vector', binary=False)